<a href="https://colab.research.google.com/github/sandeep0412/Comparing-Airbnb-Price-Prediction-Models/blob/master/Airbnb_Preliminaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Preparation

In [0]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import collections

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
%matplotlib inline
import scipy.stats as stats
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn import linear_model


In [0]:
#Amsterdam
#!wget  -O /content/drive/My\ Drive/Colab\ Notebooks/listings_a.csv.gz "http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2018-11-04/data/listings.csv.gz"
#!gunzip /content/drive/My\ Drive/Colab\ Notebooks/listings_a.csv.gz
listings=pd.read_csv("/content/drive/My Drive/Colab Notebooks/listings_a.csv")
#listings.info()
#listings.head()
#listings.describe()

#!wget  -O /content/drive/My\ Drive/Colab\ Notebooks/reviews.csv.gz "http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2018-10-05/data/reviews.csv.gz"
#!gunzip /content/drive/My\ Drive/Colab\ Notebooks/reviews.csv.gz
reviews=pd.read_csv("/content/drive/My Drive/Colab Notebooks/reviews.csv")
#reviews.info()
#reviews.describe()

#!wget  -O /content/drive/My\ Drive/Colab\ Notebooks/calendar_a.csv.gz "http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2018-11-04/data/calendar.csv.gz"
#!gunzip /content/drive/My\ Drive/Colab\ Notebooks/calendar_a.csv.gz
calendar=pd.read_csv("/content/drive/My Drive/Colab Notebooks/calendar_a.csv")
#calendar.info()
#calendar.head()
#calendar.describe()

In [0]:
#New York
#!wget  -O /content/drive/My\ Drive/Colab\ Notebooks/listings.csv.gz "http://data.insideairbnb.com/united-states/ny/new-york-city/2018-11-03/data/listings.csv.gz"
#!gunzip /content/drive/My\ Drive/Colab\ Notebooks/listings.csv.gz
#listings=pd.read_csv("/content/drive/My Drive/Colab Notebooks/listings.csv")
#listings.info()
#listings.head()
#listings.describe()

#!wget  -O /content/drive/My\ Drive/Colab\ Notebooks/reviews.csv.gz "http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2018-10-05/data/reviews.csv.gz"
#!gunzip /content/drive/My\ Drive/Colab\ Notebooks/reviews.csv.gz
#reviews=pd.read_csv("/content/drive/My Drive/Colab Notebooks/reviews.csv")
#reviews.info()
#reviews.describe()

In [0]:
#Barcelona
#!wget  -O /content/drive/My\ Drive/Colab\ Notebooks/listings.csv.gz "http://data.insideairbnb.com/spain/catalonia/barcelona/2018-11-07/data/listings.csv.gz"
#!gunzip /content/drive/My\ Drive/Colab\ Notebooks/listings.csv.gz
#listings=pd.read_csv("/content/drive/My Drive/Colab Notebooks/listings.csv")
#listings.info()
#listings.head()
#listings.describe()

#!wget  -O /content/drive/My\ Drive/Colab\ Notebooks/reviews.csv.gz "http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2018-10-05/data/reviews.csv.gz"
#!gunzip /content/drive/My\ Drive/Colab\ Notebooks/reviews.csv.gz
#reviews=pd.read_csv("/content/drive/My Drive/Colab Notebooks/reviews.csv")
#reviews.info()
#reviews.describe()

In [0]:
list(listings)

In [0]:
list(reviews)

In [0]:
list(calendar)

In [0]:
def percent_empty(df):
    
    bools = df.isnull().tolist()
    percent_empty = float(bools.count(True)) / float(len(bools))
    
    return percent_empty, float(bools.count(True))

# Store emptiness for all features
emptiness = []

missing_columns = []

# Get emptiness for all features
for i in range(0, listings.shape[1]):
    p, n = percent_empty(listings.iloc[:,i])
    if n > 0:
        missing_columns.append(listings.columns.values[i])
    emptiness.append(round((p), 2))
    
empty_dict = dict(zip(listings.columns.values.tolist(), emptiness))

# Create a temporary copy of dictionary
copyOfDict = dict(empty_dict)
 
# Iterate over the temporary dictionary and delete corresponding key from original dictionary
for (key, value) in copyOfDict.items() :
    if value % 3 == 0:
        del empty_dict[key]

# Plot emptiness graph
empty = pd.DataFrame.from_dict(empty_dict, orient = 'index').sort_values(by=0)
ax = empty.plot(kind = 'bar', color='#E35A5C', figsize = (16, 5))
ax.set_xlabel('Predictor')
ax.set_ylabel('Percent Empty / NaN')
ax.set_title('Feature Emptiness')
ax.legend_.remove()

plt.show()

In [0]:
listings_clean = listings[['id','host_id','property_type','room_type', 'accommodates','beds', 'bedrooms',
                           'bed_type', 'price','number_of_reviews','review_scores_rating','host_listings_count',
                           'minimum_nights','bathrooms', 'availability_30', 'availability_60', 'availability_365', 'latitude','longitude','neighbourhood']]
listings_clean.head()
listings_clean.isna().sum()


In [0]:
listings_clean = listings_clean.dropna(how='any',subset=['host_listings_count', 'bathrooms', 'neighbourhood'])

#listings_clean.zipcode = listings_clean.zipcode.str.replace(r'-\d+', '')
listings_clean.beds = listings_clean.beds.fillna(0)
listings_clean.bedrooms = listings_clean.bedrooms.fillna(0)

listings_clean.price = listings_clean.price.apply(lambda x: x.split('.')[0]).replace('[^0-9]', '', regex=True).apply(lambda x: int(x))
listings_clean = listings_clean[listings_clean['price'] != 0.00]
listings_clean = listings_clean[listings_clean['bedrooms'] != 0]
listings_clean = listings_clean[listings_clean['beds'] != 0]
listings_clean = listings_clean[listings_clean['bathrooms'] != 0]



In [0]:
listings_clean.isna().sum()

In [0]:
idx_vals = listings_clean['review_scores_rating'][listings_clean['number_of_reviews'] == 0].index.values.tolist()
#print(idx_vals)
listings_clean.loc[idx_vals, ('review_scores_rating')] = listings_clean['review_scores_rating'][listings_clean['number_of_reviews'] == 0].replace('NaN', 'No Reviews')
listings_clean = listings_clean[~listings_clean['review_scores_rating'].isnull()]
#print(np.unique(listings_clean['review_scores_rating']))
print(np.unique(listings_clean['neighbourhood']))
listings_clean['neighbourhood'].nunique()


In [0]:
# Get number of listings in neighborhoods
nb_counts = collections.Counter(listings.neighbourhood)
tdf = pd.DataFrame.from_dict(nb_counts, orient='index').sort_values(by=0)

# Plot number of listings in each neighborhood
ax = tdf.plot(kind='bar', figsize = (20,10), color = 'r', alpha = 0.85)
ax.set_title("Neighborhoods by Number of Listings")
ax.set_xlabel("Neighborhood")
ax.set_ylabel("# of Listings")
plt.show()

In [0]:
# Delete neighborhoods with less than 100 entries
del_list=[]
for i in nb_counts.keys():
    if nb_counts[i] < 100:
      del_list.append(nb_counts[i])
      listings_clean = listings_clean[listings_clean.neighbourhood!= i]

for i in del_list:
  del nb_counts[i]
print(listings_clean['neighbourhood'].nunique())
# Plot new neighborhoods distribution
tdf = pd.DataFrame.from_dict(nb_counts, orient='index').sort_values(by=0)
ax = tdf.plot(kind='bar', figsize = (22,4), color = 'r', alpha = 0.85)
ax.set_title("Neighborhoods by House # (Top 48)")
ax.set_xlabel("Neighborhood")
ax.set_ylabel("# of Listings")

plt.show()
entries = listings_clean.shape[0]
print (entries - listings_clean.shape[0])
entries = listings_clean.shape[0]



## Data Exploration

In [0]:

fig, ax = plt.subplots(1, figsize = (8, 7.5))
d, bins, patches = ax.hist(listings_clean.price, 100, alpha = 0.85)
ax.set_title("Histogram of Price Distribution - All prices")
ax.set_xlabel("Daily Listing Price in Dollars")
ax.set_ylabel("Frequency")
plt.show()




In [0]:
grp_price_500 = np.arange(min(listings_clean.price),500,1)
listings_price_500 = pd.DataFrame({'id':listings_clean['id'], 'price':np.digitize(listings_clean.price, grp_price_500)})
listings_price_500.price.plot(kind='hist')
plt.title('Price distribution 0 - $200')
plt.show()

In [0]:
grp_price_100 = np.arange(min(listings_clean.price),100,1)
listings_price_100 = pd.DataFrame({'id':listings_clean['id'], 'price':np.digitize(listings_clean.price, grp_price_100)})
listings_price_100.price.plot(kind='hist')
plt.title('Price distribution 0 - $100')
plt.show()

In [0]:
#Neighbourhood prices
df_nprice = pd.DataFrame({'neighbourhood':listings_clean['neighbourhood'], 'price': listings_clean['price']})
df_nprice.groupby('neighbourhood')['price'].mean().plot.bar()
#print(df_nprice)

In [0]:
import collections

# Get frequency of bedroom number for listings
bedrooms_counts = collections.Counter(listings_clean.bedrooms)
tdf = pd.DataFrame.from_dict(bedrooms_counts, orient = 'index').sort_values(by = 0)
tdf = (tdf.iloc[-10:, :] / 27392) * 100

# Sort bedroom dataframe by number
tdf.sort_index(axis = 0, ascending = True, inplace = True)

# Plot percent of listings by bedroom number
ax = tdf.plot(kind = 'bar', figsize = (12, 4), alpha = 0.85)
ax.set_xlabel("Number of Bedrooms")
ax.set_ylabel("Percent of Listings")
ax.set_title('Percent of Listings by Bedrooms')
ax.legend_.remove()

plt.show()


In [0]:
# Outline price buckets
intervals = [0, 100, 300, 10000]
leg_labels = []

# Get Labels for legend
for i in range(0,len(intervals) - 1):
    if i == len(intervals) - 2:
        leg_labels.append('\${}+'.format(intervals[i]))
    else:
        leg_labels.append("\${}-\${}".format(intervals[i], intervals[i+1]))    

buckets = []
BNB_BLUE = '#007A87'
BNB_RED = '#FF5A5F'
BNB_DARK_GRAY = '#565A5C'
BNB_LIGHT_GRAY = '#CED1CC'

# Divide up into price buckets
for i in range(0, len(intervals) - 1):
    buckets.append(listings_clean[(listings_clean.price > intervals[i]) & (listings_clean.price < intervals[i+1])])

colors = ['#007A87', '#FF5A5F', 'g']
alphas = [0.85, 0.85, 0.85]  

# Plot listings on scatterplot
plt.figure(figsize=(15, 15))
for i in range(0, len(buckets)):
    plt.scatter(buckets[i]['longitude'], buckets[i]['latitude'], c=colors[i], alpha = alphas[i], s=25)
    
plt.title('Amsterdam Airbnb Listings')
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.legend(labels=leg_labels, loc = 'best')
#plt.xlim(34.2,73.7)
#plt.ylim(40.45,40.95)

plt.show()

In [0]:
# encode categorical variables
neighbourhood_dummies = pd.get_dummies(listings_clean['neighbourhood'])
property_dummies = pd.get_dummies(listings_clean['property_type'])
room_dummies = pd.get_dummies(listings_clean['room_type'])
bed_dummies = pd.get_dummies(listings_clean['bed_type'])
#ratings_scores_dummies = pd.get_dummies(listings_clean['review_scores_rating'])

# replace the old columns with our new one-hot encoded ones
df = pd.concat((listings_clean.drop(['neighbourhood', \
    'property_type', 'room_type', 'bed_type'], axis=1), \
    neighbourhood_dummies.astype(int), property_dummies.astype(int), \
    room_dummies.astype(int), bed_dummies.astype(int)), \
    axis=1)

print(len(df.columns))

# move target predictor 'price' to the end of the listings_clean frame
cols = list(df.columns.values)
idx = cols.index('price')
rearrange_cols = cols[:idx] + cols[idx+1:] + [cols[idx]]
df = df[rearrange_cols]

In [0]:
# convert non-categorical variables to floats and standardize
def standardize_col(col):
    mean = np.mean(col)
    std = np.std(col)
    return col.apply(lambda x: (x - mean) / std)

non_cat_vars = ['accommodates', 'bedrooms', 'beds', 'number_of_reviews', 'host_listings_count', 'availability_30', 'minimum_nights', 'bathrooms', 'review_scores_rating']
for col in non_cat_vars:
    df[col] = df[col].astype(float)
    df[col] = standardize_col(df[col])
    
df.head()

In [0]:
import math
# log transform the response 'price'
print(min(df.price))
df['price_log'] = df['price'].apply(lambda x: math.log(x))

In [0]:
import pylab
stats.probplot(df['price_log'], dist="norm", plot=pylab)
pylab.show()

In [0]:

# read to csv
df.to_csv('/content/drive/My Drive/Colab Notebooks/listings_clean_a1.csv', index=False)

In [0]:
df.shape

In [0]:
# check for collinearity and visualize relationship between each predictor and price
matrix_cols = ['accommodates', 'bedrooms', 'beds', 'minimum_nights', 'availability_30', 'availability_60', 'availability_365', 'number_of_reviews', 'host_listings_count', 'review_scores_rating', 'price']
axs = pd.plotting.scatter_matrix(df[matrix_cols], figsize=(20, 20), c='red')

In [0]:
# Compute matrix of correlation coefficients
corr_matrix = np.corrcoef(df.T)

corr_df = pd.DataFrame(data = corr_matrix, columns = df.columns, 
             index = df.columns)
# Display heat map 
plt.figure(figsize=(7, 7))
plt.pcolor(corr_matrix, cmap='RdBu')
plt.xlabel('Predictor Index')
plt.ylabel('Predictor Index')
plt.title('Heatmap of Correlation Matrix')
plt.colorbar()

plt.show()

##Seasonality


In [0]:
calendar=pd.read_csv("/content/drive/My Drive/Colab Notebooks/calendar_a.csv")
list(calendar)

In [0]:
calendar.isna().sum()

In [0]:
calendar = calendar.dropna()
calendar.price = calendar.price.apply(lambda x: x.split('.')[0]).replace('[^0-9]', '', regex=True).apply(lambda x: int(x))
calendar = calendar[calendar['price'] != 0.00]

In [0]:
calendar.shape

In [0]:
calendar.dtypes

In [0]:
month = []
for index, row in calendar.iterrows():
  month.append(int(row['date'].split('-')[1]))
calendar['month'] = month
calendar.head(5)


In [0]:
calendar.shape

In [0]:
season_prices = calendar.groupby(['listing_id', 'month'],  as_index=False )['price'].mean()
#season_prices = calendar.groupby(['listing_id', 'month'],  as_index=False )['price'].median()

In [0]:
season_prices.dtypes

In [0]:
listings_clean.id.nunique()

In [0]:
id_list = season_prices['listing_id'].unique()

In [0]:
listings_clean.dtypes

In [0]:
season_price = []
for index, row in listings_clean.iterrows():
  if row['id'] in id_list:
    season_price.append(season_prices['price'][season_prices['listing_id'] == row['id']])
  else:
    season_price.append(0)
ser = np.empty_like(season_price, dtype='float64')
listings_clean['season'] = ser

In [0]:
df['season'] = listings_clean['season']

In [0]:
df.dtypes

In [0]:
df.to_csv('/content/drive/My Drive/Colab Notebooks/listings_clean_as1.csv', index=False)

##Reviews Sentiment Analysis

In [0]:
reviews.head(n=10)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...
5,2818,4748,2009-06-29,20192,Jie,Daniel was a great host! He made everything so...
6,2818,5202,2009-07-07,23055,Vanessa,Daniele is an amazing host! He provided everyt...
7,2818,9131,2009-09-06,26343,Katja,You can´t have a nicer start in Amsterdam. Dan...
8,2818,12103,2009-10-01,40999,Marie-Eve,Daniel was a fantastic host. His place is calm...
9,2818,16196,2009-11-04,38623,Graham,Daniel was great. He couldn.t do enough for us...


In [0]:
print ('Reviews before NaNs dropped: ', reviews.shape[0])
# Delete reviews entries with NaN values (empty review)
reviews = reviews.dropna(axis=0)
print ('Reviews after NaNs dropped: ', reviews.shape[0])

Reviews before NaNs dropped:  421031
Reviews after NaNs dropped:  420487


In [0]:
# creates empty array to store concatenated text
empty_concat = []

# iterates through each listing id
for item in listings_unclean['id'].unique():
    # checks to see if the corresponding listing id has a review
    # if so it concatenates the listing title with the reivew
    if item in reviews['listing_id'].unique():
        empty_concat.append(str(listings_unclean['name'].loc[listings_unclean['id'] == item].values[-1]) + ' ' + str(reviews['comments'].loc[reviews['listing_id'] == item].values[:-1]))
    else:
        empty_concat.append(str(listings_unclean['name'].loc[listings_unclean['id'] == item].values[-1]))

In [0]:
neighborhood_words = []

for item in listings_unclean['id'].values:
    neighborhood_words.append(listings_unclean['neighbourhood'].loc[listings_unclean['id'] == item].values[-1])

listings_unclean['neighborhood_names'] = neighborhood_words

In [0]:
listings_unclean.info()

In [0]:
print(listings_unclean['neighborhood_names'].unique().tolist())

In [0]:
# Get Neighborhood words to remove geographic correlation from our 'bag-of-wods'
nbs = listings_unclean['neighborhood_names'].unique().tolist()
nbs2=[]
for nb in nbs:
  if type(nb)==str:
    nbs2.append(nb.lower().split())
nbs = set([item for sublist in nbs2 for item in sublist])

In [0]:
import re
 import nltk
 nltk.download('stopwords')

In [0]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')

from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

bags = empty_concat
bag_of_words = []

# Make characters all english alphabet
for review in bags:
    # Use regular expressions to do a find-and-replace for alphabet
    replaced = re.sub("[^a-zA-Z]", " ", review)
        
    # All lower
    lower_case = replaced.lower()
    
    # Put in list
    words = lower_case.split()
    
    # Exclude stop-words and neighborhood words
    words = [stemmer.stem(w) for w in words if ((not w in stops) & (not w in nbs))]
    listing_words = ' '.join(words)     
    bag_of_words.append(listing_words)

In [0]:
# Output bag_of_words feature to csv
bag = pd.DataFrame(listings_unclean['id'])
bag['bag_of_words'] = bag_of_words
bag.to_csv('bag_of_words.csv')

bag.head(n=10)

In [0]:
!pip install twython
nltk.download('vader_lexicon')

In [0]:
full_bag = bag['bag_of_words'].tolist()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

nltk_polarities = []

for each in full_bag:
    each = " ".join(each)
    ss = sid.polarity_scores(each)
    nltk_polarities.append(ss.values())
print ('Polarities appended in form [neg, neu, pos, compound]')

In [0]:
!pip install textblob

In [0]:
from textblob import TextBlob

tb_polarities = []

for i, each in enumerate(full_bag):
    blob = TextBlob(each)
    tb_polarities.append(blob.sentiment.polarity)

In [0]:
print(len(tb_polarities))
listings_unclean.shape

In [0]:
# Initiate Figure
ax = plt.axes()
BINS = 30

# Plot Text Blob Polarity Average Distribution
ax.hist(tb_polarities, bins = BINS, color = BNB_BLUE)
ax.set_title('Distribution of TextBlob Polarity Averages')
ax.set_xlim([0,1])
ax.set_ylim([0,9000])

plt.show()

In [0]:
import collections
# Flatten List
list_bag = [w.split() for w in full_bag]
all_words = [item for sublist in list_bag for item in sublist]

# Get 1000 most common tokens
most_common = collections.Counter(all_words).most_common(1000)

# Remove words shorter than length 2
common_words = [w for w in most_common if len(w[0]) > 2]
ax = plt.figure(111, figsize = (10,4))

plt.bar(range(len(common_words)), [w[1] for w in common_words], align='center', color = BNB_BLUE)
plt.xlim([0,1000])
plt.title('Most Common Tokens Distribution')
plt.xlabel('Words')
plt.ylabel('Frequency')

plt.show()

In [0]:
# Outline price buckets
intervals = [0,100,300,10000]
leg_labels = []

# Get Labels for Legend
for i in range(0,len(intervals) - 1):
    if i == len(intervals) - 2:
        leg_labels.append('\${}+'.format(intervals[i]))
    else:
        leg_labels.append("\${}-\${}".format(intervals[i], intervals[i+1]))    
listings_season=pd.read_csv("/content/drive/My Drive/listings_clean_a.csv")
buckets = []
x = listings_season#.drop('id', 1)

# Divide up into buckets
for i in range(0, len(intervals) - 1):
    buckets.append(x[(x['price'] > intervals[i]) & (x['price'] < intervals[i+1])])

In [0]:
calendar.price.fillna(0, inplace=True)
calendar.isna().sum()

In [0]:
import re
prices=[]
calendar = calendar[calendar.price != 0]
for p in calendar.price:
    p = re.sub('[^0-9.]+','', p)
    prices.append(float(p))
    
calendar.price = prices


In [0]:

# creates empty array to store the listing IDs of into price buckets
high_bucket = []
mid_bucket = []
low_bucket = []

# iterates through each row in the calendar dataframe
for index, item in calendar.iterrows():
    # checks to see which price bucket our listing IDs fall in 
    if item['listing_id'] in buckets[0]['id'].unique():
        low_bucket.append(item)
    elif item['listing_id'] in buckets[1]['id'].unique():
        mid_bucket.append(item)
    elif item['listing_id'] in buckets[2]['id'].unique():
        high_bucket.append(item)

In [0]:
from datetime import datetime
# crtes a calendar dates array consisting of all the unique calendar dates (366)
calendar_dates = np.array(calendar['date'].unique())
calendar_dates.sort()

# converts arrays to pandas dataframes
high_bucket = pd.DataFrame(high_bucket)
mid_bucket = pd.DataFrame(mid_bucket)
low_bucket = pd.DataFrame(low_bucket)

# empty array to store the average daily price information based on price bucket
high_seasonality_data = np.empty([len(calendar_dates),1])
mid_seasonality_data = np.empty([len(calendar_dates),1])
low_seasonality_data = np.empty([len(calendar_dates),1])

index = 0

# iterates each day of the calendar
for item in calendar_dates:
    calendar_dates[index] = datetime.strptime(item,'%Y-%m-%d').date()
    high_seasonality_data[index, 0] = np.mean(high_bucket['price'].loc[high_bucket['date'] == item])
    mid_seasonality_data[index, 0] = np.mean(mid_bucket['price'].loc[mid_bucket['date'] == item])
    low_seasonality_data[index, 0] = np.mean(low_bucket['price'].loc[low_bucket['date'] == item])
    index += 1

In [0]:
plt.figure(figsize=(10, 5))

# prints all buckets (high, mid, and low) containing seasonality data
plt.plot(calendar_dates, high_seasonality_data, label = 'High Price Bucket', color = BNB_RED)
plt.plot(calendar_dates, mid_seasonality_data, label = 'Middle Price Bucket', color = BNB_BLUE)
plt.plot(calendar_dates, low_seasonality_data, label = 'Low Price Bucket', color = BNB_DARK_GRAY)

plt.title('Average Daily Price')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend(loc = 'best')

plt.show()

In [0]:
import matplotlib
matplotlib.pyplot.scatter(calendar_dates, high_seasonality_data)

In [0]:
# plots only listings in the high price bucket
plt.figure(figsize=(10, 5))

plt.plot(calendar_dates, high_seasonality_data, color = BNB_RED, label = 'Average Daily Price')
plt.plot(calendar_dates, np.median(high_seasonality_data)*np.ones([len(high_seasonality_data),1]), color = BNB_BLUE, label = 'Median Daily Price')

plt.title('High Price Bucket')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend(loc = 'best')
# plt.ylim(460,580)

plt.show()

In [0]:
# plots only listings in the middle price bucket
plt.figure(figsize=(10, 5))

plt.plot(calendar_dates, mid_seasonality_data, color = BNB_RED ,label = 'Average Daily Price')
plt.plot(calendar_dates, np.median(mid_seasonality_data)*np.ones([len(mid_seasonality_data),1]), color = BNB_BLUE, label = 'Median Daily Price')

plt.title('Middle Price Bucket')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend(loc = 'best')

plt.show()

In [0]:
# plots only listings in the low price bucket
plt.figure(figsize=(10, 5))

plt.plot(calendar_dates, low_seasonality_data, color = BNB_RED ,label = 'Average Daily Price')
plt.plot(calendar_dates, np.median(low_seasonality_data)*np.ones([len(low_seasonality_data),1]), color = BNB_BLUE, label = 'Median Daily Price')

plt.title('Low Price Bucket - Average Nightly Price')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend(loc = 'best')
# plt.ylim(70,90)

plt.show()

In [0]:
len(tb_polarities)

In [0]:
listings_unclean.shape

In [0]:
listings_season.shape

In [0]:
# exports transit location / distance information to a csv
exported_data = pd.DataFrame({'id': listings_unclean['id'].values, 'polarities': tb_polarities})
exported_data = exported_data[['id', 'polarities']]


exported_data.head(n=5)

In [0]:
temp=pd.merge(exported_data, listings, how='inner', on='id')

In [0]:
listings=temp